In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_Union_Councils_of_Karachi').text
soup = BeautifulSoup(source,'lxml')
uc_tab = soup.find('div', class_='toc').ul
district = []
for lis in uc_tab.find_all('li',class_='toclevel-2'):
    district.append(lis.find('span', class_='toctext').text)

In [4]:
district

['Baldia Town',
 'Bin Qasim Town',
 'Gadap Town',
 'Gulberg Town',
 'Gulshan Town',
 'Jamshed Town',
 'Kemari Town',
 'Korangi Town',
 'Landhi Town',
 'Liaquatabad Town',
 'Lyari Town',
 'Malir Town',
 'New Karachi Town',
 'North Nazimabad Town',
 'Orangi Town',
 'Saddar Town',
 'Shah Faisal Town',
 'S.I.T.E. Town (Sindh Industrial & Trading Estate)']

## for town

In [5]:
town = []
districts = [] 
dist_and_town = {}
for dis_links in district:
    source2 = requests.get('https://en.wikipedia.org/wiki/{}'.format(dis_links)).text
    soup2 = BeautifulSoup(source2,'lxml')
    try:
        neigh_tab = soup2.find('div',class_='div-col columns column-width')
        for lis2 in neigh_tab.find_all('li'):
            town.append(lis2.text)
            districts.append(dis_links)
    except Exception as e:
        pass

dist_and_town = {'Districts':districts,'Town':town}

## create dataframe

In [6]:
df_khi = pd.DataFrame(dist_and_town)
df_khi.head()

,Districts,Town
0,Bin Qasim Town,Abdullah Goth
1,Bin Qasim Town,Cattle Colony
2,Bin Qasim Town,Gaghar
3,Bin Qasim Town,Green Park City
4,Bin Qasim Town,Gulshan-e-Hadeed


In [7]:
df_khi.shape

(220, 2)

**Some Neighborhoods enter manually**

In [8]:
#save the file sp enter manually
#df_khi.to_csv('karachi.csv')

In [10]:
#read file
df_khi = pd.read_csv('karachi_district.csv')
df_khi.head()

,Districts,Town
0,Bin Qasim Town,Abdullah Goth
1,Bin Qasim Town,Cattle Colony
2,Bin Qasim Town,Gaghar
3,Bin Qasim Town,Green Park City
4,Bin Qasim Town,Gulshan-e-Hadeed


In [11]:
df_khi.shape

(294, 2)

In [12]:
import geocoder
from geopy.geocoders import Nominatim

**Find the Co-ordiantes of each Neighborhood**

In [ ]:
latitude = []
longitude = []
for t_loc in df_khi['Town']: 
    address = '{},Karachi,Pakistan'.format(t_loc)
    geolocator = Nominatim(user_agent='khi_explorer')
    location = geolocator.geocode(address,timeout=1000)
    try:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
        print('Latitude {} and Longitude {}'.format(latitude,longitude))
    except Exception as e:
        latitude.append("")
        longitude.append("")

In [99]:
len(latitude)

294

In [100]:
len(longitude)

294

In [101]:
df_khi['Latitude'] = latitude
df_khi['Longitude'] = longitude
df_khi.head()

,Districts,Town,Latitude,Longitude
0,Bin Qasim Town,Abdullah Goth,24.8665,67.2837
1,Bin Qasim Town,Cattle Colony,24.8793,67.1987
2,Bin Qasim Town,Gaghar,,
3,Bin Qasim Town,Green Park City,24.8577,67.2225
4,Bin Qasim Town,Gulshan-e-Hadeed,24.87,67.3601


In [102]:
df_khi.to_csv('karachi_lat_lng.csv')

In [106]:
len(df_khi[df_khi['Latitude'] == ''])

81

In [121]:
dfr = df_khi.drop(df_khi[df_khi['Latitude'] == ""].index,axis=0)

In [123]:
len(dfr[dfr['Latitude'] == ''])

0

In [13]:
df_khi = pd.read_csv('karachi_lat_lng.csv')
df_khi.head()

,Districts,Town,Latitude,Longitude
0,Bin Qasim Town,Abdullah Goth,24.866540,67.283672
1,Bin Qasim Town,Cattle Colony,24.879311,67.198723
2,Bin Qasim Town,Green Park City,24.857659,67.222475
3,Bin Qasim Town,Gulshan-e-Hadeed,24.869999,67.360052
4,Bin Qasim Town,Ibrahim Hyderi,24.791608,67.140916


In [15]:
df_khi.shape

(213, 4)

## folium

In [38]:
import folium
from pandas.io.json import json_normalize

In [18]:
map_khi = folium.Map(
    location=[24.9008, 67.1681], 
    zoom_start=10
)

In [19]:
for lat, lng, borough, neighborhood in zip(df_khi['Latitude'], df_khi['Longitude'], df_khi['Districts'], df_khi['Town']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_khi)  
    
map_khi

In [20]:
CLIENT_ID = 'KAJZOUZF20UXDL2H3MOEBLTIHGQ2HK1PDJ14Z1R0QCAJCYVA' # your Foursquare ID
CLIENT_SECRET = 'UOJCAODP5HOBJS1W451QQ3JQ3HSAWPQ2TL2WWMCJXLFAYD33' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KAJZOUZF20UXDL2H3MOEBLTIHGQ2HK1PDJ14Z1R0QCAJCYVA
CLIENT_SECRET:UOJCAODP5HOBJS1W451QQ3JQ3HSAWPQ2TL2WWMCJXLFAYD33


In [24]:
df_khi[df_khi['Town'] == 'Clifton'].index

Int64Index([202], dtype='int64')

In [25]:
neighborhood_latitude = df_khi['Latitude'][202] # neighborhood latitude value
neighborhood_longitude = df_khi['Longitude'][202] # neighborhood longitude value

neighborhood_name = df_khi['Town'][202] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Clifton are 24.8190552, 67.0262397.


In [33]:

LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 1000 # define radius




url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KAJZOUZF20UXDL2H3MOEBLTIHGQ2HK1PDJ14Z1R0QCAJCYVA&client_secret=UOJCAODP5HOBJS1W451QQ3JQ3HSAWPQ2TL2WWMCJXLFAYD33&v=20180605&ll=24.8190552,67.0262397&radius=1000&limit=100'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e809bce14a12661731c7cce'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Clifton',
  'headerFullLocation': 'Clifton, Karachi',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 40,
  'suggestedBounds': {'ne': {'lat': 24.82805520900001, 'lng': 67.036137044161},
   'sw': {'lat': 24.810055190999993, 'lng': 67.01634235583901}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b5fdbf6f964a520f9ce29e3',
       'name': 'Bar-B-Q Tonight',
       'location': {'address': 'Com 5/1, Sh-e-Firdousi, Boat Basin, Clifton Block-5',
        'crossStreet': 'Khayaban-e-Saadi, opp. Bilawal Chowrangi',
        'lat': 24.816200703197044,
        'lng

In [35]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

c:\users\aitezaz akhtar\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Bar-B-Q Tonight,BBQ Joint,24.816201,67.021181
1,Mohatta Palace,History Museum,24.814550,67.032652
2,Karachi Broast,Fast Food Restaurant,24.826819,67.026328
3,Tao - Pan Asian Cuisine,Asian Restaurant,24.827452,67.027946
4,Boat Basin,Street Food Gathering,24.826821,67.025774


In [37]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

40 venues were returned by Foursquare.
